In [17]:
#! pip install pandas
#! pip install chardet

In [18]:
import requests
import io
import pandas as pd
import zipfile
import os

#### Extraindo os últimos dados públicos disponíveis de viagens no Portal da Transparência

In [19]:

# URL of the zip file to be downloaded
url = 'https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/viagens/2024_20231122_Viagens.zip'

# Perform a GET request to retrieve the zip file
response = requests.get(url)

if response.status_code == 200:
    # Find the download link from the response (assuming it redirects to the zip file)
    download_url = response.url

    # Download the zip file
    zip_response = requests.get(download_url)
    
    if zip_response.status_code == 200:
        # Create a BytesIO object to work with the content as a file-like object
        zip_file = io.BytesIO(zip_response.content)
        
        # Extract the contents of the zip file
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            # Extract all contents to a specified directory
            zip_ref.extractall('../data/TPM1')  # Replace '/path/to/extract' with your desired directory
            print("Extraction complete.")
    else:
        print("Failed to download the zip file.")
else:
    print("Failed to fetch the download URL.")


Extraction complete.


#### Listando os arquivos baixados no diretório data

In [20]:
os.listdir('../data/TPM1')

['2024_viagens_consolidado.csv',
 '2024_Viagem.csv',
 '2024_Passagem.csv',
 '2024_Pagamento.csv',
 '.~lock.2024_viagens_consolidado.csv#',
 '2024_Trecho.csv']

#### carregando os dataframes com pandas

In [21]:
pagamento = pd.read_csv('../data/TPM1/2024_Pagamento.csv', index_col=False, encoding='ISO-8859-1', delimiter=';')
passagem = pd.read_csv('../data/TPM1/2024_Passagem.csv', index_col=False, encoding='ISO-8859-1', delimiter=';')
trecho = pd.read_csv('../data/TPM1/2024_Trecho.csv', index_col=False, encoding='ISO-8859-1', delimiter=';')
viagem = pd.read_csv('../data/TPM1/2024_Viagem.csv', index_col=False, encoding='ISO-8859-1', delimiter=';')

In [22]:
#pagamento.info()
#viagem.shape

#### mesclando os dataframes usando a coluna 'Identificador do processo de viagem'

In [23]:
df = pd.merge(pagamento, passagem, left_index=True, right_index=True, suffixes=('_pagamento', '_passagem'), how='inner')
df = pd.merge(df, trecho, left_index=True, right_index=True, suffixes=('_merged', '_trecho'), how='inner')
df = pd.merge(df, viagem, left_index=True, right_index=True, suffixes=('_final', '_viagem'), how='inner')

In [24]:
df.to_csv('../data/TPM1/2024_viagens_consolidado.csv', index=False)